# Кодировка свечей по методу Лиховидова, прогноз Up, Flat, Down.

Обучение модели на основе кода Лиховидова.  
Тройная классификация Up, Flat, Down.

In [64]:
# # используются фиксированные пороги (0.005 и -0.005).
# import sqlite3
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import numpy as np
# import pandas as pd
# from pathlib import Path
# from torch.utils.data import Dataset, DataLoader

# # === 1. ЗАГРУЗКА ДАННЫХ ===
# db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

# with sqlite3.connect(db_path) as conn:
#     df_fut = pd.read_sql_query(
#         "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Futures",
#         conn
#     )

# # === 2. КОДИРОВАНИЕ СВЕЧЕЙ (ЛИХОВИДОВ) ===
# def encode_candle(row):
#     open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    
#     direction = 1 if close > open_ else (0 if close < open_ else 2)
#     upper_shadow = high - max(open_, close)
#     lower_shadow = min(open_, close) - low
#     body = abs(close - open_)

#     def classify_shadow(shadow, body):
#         return 0 if shadow < 0.1 * body else (1 if shadow < 0.5 * body else 2)

#     return f"{direction}{classify_shadow(upper_shadow, body)}{classify_shadow(lower_shadow, body)}"

# df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# # === 3. ПОДГОТОВКА ДАННЫХ ДЛЯ LSTM ===
# unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
# code_to_int = {code: i for i, code in enumerate(unique_codes)}
# df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

# window_size = 20  
# predict_offset = 1  

# X, y = [], []
# for i in range(len(df_fut) - window_size - predict_offset):
#     X.append(df_fut['CANDLE_INT'].iloc[i:i+window_size].values)
    
#     future_price = df_fut['CLOSE'].iloc[i+window_size+predict_offset]
#     current_price = df_fut['CLOSE'].iloc[i+window_size]
#     change = (future_price - current_price) / current_price  

#     # Add target
#     if change > 0.005:  # Рост > 0.5%
#         y.append(2)  # +1 (сильный рост)
#     elif change < -0.005:  # Падение > 0.5%
#         y.append(0)  # -1 (сильное падение)
#     else:
#         y.append(1)  # 0 (флэт)

# X, y = np.array(X), np.array(y)

# split = int(0.8 * len(X))
# X_train, y_train = X[:split], y[:split]
# X_test, y_test = X[split:], y[split:]

# class CandlestickDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = torch.tensor(X, dtype=torch.long)
#         self.y = torch.tensor(y, dtype=torch.long)  # Теперь метки - целые числа (0, 1, 2)

#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# train_dataset = CandlestickDataset(X_train, y_train)
# test_dataset = CandlestickDataset(X_test, y_test)

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# # === 4. LSTM-МОДЕЛЬ С 3-КЛАССОВОЙ КЛАССИФИКАЦИЕЙ ===
# class CandleLSTM(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
#         super(CandleLSTM, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         x = self.embedding(x)
#         x, _ = self.lstm(x)
#         x = self.fc(x[:, -1, :])  
#         return self.softmax(x)  # Выдает вероятности 3 классов

# # === 5. ОБУЧЕНИЕ С СОХРАНЕНИЕМ ЛУЧШЕЙ МОДЕЛИ ===
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model = CandleLSTM(
#     vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=3
#     ).to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# best_accuracy = 0  
# model_path = "best_model_04.pth"

# epochs = 2000
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for X_batch, y_batch in train_loader:
#         X_batch, y_batch = X_batch.to(device), y_batch.to(device)

#         optimizer.zero_grad()
#         y_pred = model(X_batch)
#         loss = criterion(y_pred, y_batch)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     # Проверка на тесте
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for X_batch, y_batch in test_loader:
#             X_batch, y_batch = X_batch.to(device), y_batch.to(device)
#             y_pred = model(X_batch).argmax(dim=1)  # Класс с наибольшей вероятностью
#             correct += (y_pred == y_batch).sum().item()
#             total += y_batch.size(0)

#     accuracy = correct / total
#     print(
#         f"Epoch {epoch+1}/{epochs}, "
#         f"Loss: {total_loss/len(train_loader):.4f}, "
#         f"Test Accuracy: {accuracy:.2%}"
#         )

#     # Сохранение лучшей модели
#     if accuracy > best_accuracy:
#         best_accuracy = accuracy
#         torch.save(model.state_dict(), model_path)
#         print(f"✅ Model saved with accuracy: {best_accuracy:.2%}")

# # === 6. ПРЕДСКАЗАНИЕ С 3 КЛАССАМИ ===
# model.load_state_dict(torch.load(model_path))
# model.eval()

# last_sequence = torch.tensor(
#     df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
#     ).unsqueeze(0).to(device)

# with torch.no_grad():
#     probabilities = model(last_sequence).squeeze().tolist()
#     predicted_class = np.argmax(probabilities)

#     class_names = {0: "📉 STRONG DOWN", 1: "🔹 FLAT", 2: "📈 STRONG UP"}

#     print(f"Prediction: {class_names[predicted_class]}")
#     print(
#         f"UP Probability: {probabilities[2]:.2%}, "
#         f"FLAT Probability: {probabilities[1]:.2%}, "
#         f"DOWN Probability: {probabilities[0]:.2%}"
#         )


In [65]:
# Пороги для классификации (high_threshold, low_threshold) теперь рассчитываются динамически 
# на основе 10-периодного перцентиля.
# Используется np.percentile() для определения 75-го и 25-го перцентиля последних 10 изменений 
# цены (past_changes).
# Если данных недостаточно (i < 10), используются фиксированные пороги (0.005 и -0.005).
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import random
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

# === ФИКСАЦИЯ СЛУЧАЙНЫХ ЧИСЕЛ ДЛЯ ДЕТЕРМИНИРОВАННОСТИ ===
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 
set_seed(13)  # Устанавливаем одинаковый seed

# === 1. ЗАГРУЗКА ДАННЫХ ===
db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

with sqlite3.connect(db_path) as conn:
    df_fut = pd.read_sql_query(
        "SELECT TRADEDATE, OPEN, LOW, HIGH, CLOSE, VOLUME FROM Futures",
        conn
    )

# === 2. КОДИРОВАНИЕ СВЕЧЕЙ (ЛИХОВИДОВ) ===
def encode_candle(row):
    open_, low, high, close = row['OPEN'], row['LOW'], row['HIGH'], row['CLOSE']
    
    direction = 1 if close > open_ else (0 if close < open_ else 2)
    upper_shadow = high - max(open_, close)
    lower_shadow = min(open_, close) - low
    body = abs(close - open_)

    def classify_shadow(shadow, body):
        return 0 if shadow < 0.1 * body else (1 if shadow < 0.5 * body else 2)

    return f"{direction}{classify_shadow(upper_shadow, body)}{classify_shadow(lower_shadow, body)}"

df_fut['CANDLE_CODE'] = df_fut.apply(encode_candle, axis=1)

# === 3. ПОДГОТОВКА ДАННЫХ ДЛЯ LSTM ===
unique_codes = sorted(df_fut['CANDLE_CODE'].unique())
code_to_int = {code: i for i, code in enumerate(unique_codes)}
df_fut['CANDLE_INT'] = df_fut['CANDLE_CODE'].map(code_to_int)

window_size = 20  
predict_offset = 1  

X, y = [], []
for i in range(len(df_fut) - window_size - predict_offset):
    X.append(df_fut['CANDLE_INT'].iloc[i:i+window_size].values)
    
    future_price = df_fut['CLOSE'].iloc[i+window_size+predict_offset]
    current_price = df_fut['CLOSE'].iloc[i+window_size]
    change = (future_price - current_price) / current_price  

    # Рассчитываем динамические пороги на основе 10-периодного перцентиля
    if i >= 10:
        past_changes = df_fut['CLOSE'].pct_change().iloc[i-10:i].dropna()
        high_threshold = np.percentile(past_changes, 75)  
        low_threshold = np.percentile(past_changes, 25)  
    else:
        high_threshold = 0.005  
        low_threshold = -0.005  

    # Динамическая классификация
    if change > high_threshold:
        y.append(2)  # Сильный рост
    elif change < low_threshold:
        y.append(0)  # Сильное падение
    else:
        y.append(1)  # Флэт

X, y = np.array(X), np.array(y)

split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

class CandlestickDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)  # Теперь метки - целые числа (0, 1, 2)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
def seed_worker(worker_id):
    np.random.seed(42 + worker_id)
    random.seed(42 + worker_id)

train_dataset = CandlestickDataset(X_train, y_train)
test_dataset = CandlestickDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, worker_init_fn=seed_worker)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, worker_init_fn=seed_worker)

# === 4. LSTM-МОДЕЛЬ С 3-КЛАССОВОЙ КЛАССИФИКАЦИЕЙ ===
class CandleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(CandleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  
        return self.softmax(x)  # Выдает вероятности 3 классов

# === 5. ОБУЧЕНИЕ С СОХРАНЕНИЕМ ЛУЧШЕЙ МОДЕЛИ ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CandleLSTM(
    vocab_size=len(unique_codes), embedding_dim=8, hidden_dim=32, output_dim=3
    ).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

best_accuracy = 0  
epoch_best_accuracy = 0
model_path = "best_model_03.pth"
early_stop_epochs = 200
epochs_no_improve = 0

epochs = 2000
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # === Проверка на тесте после каждой эпохи ===
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch).argmax(dim=1)  # Класс с наибольшей вероятностью
            correct += (y_pred == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total
    print(
        f"Epoch {epoch+1}/{epochs}, "
        f"Loss: {total_loss/len(train_loader):.4f}, "
        f"Test Accuracy: {accuracy:.2%}, "
        f"Best accuracy: {best_accuracy:.2%}, "
        f"Epoch best accuracy: {epoch_best_accuracy}, "
        f"seed: 1"
        )

    # Сохранение лучшей модели
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        epochs_no_improve = 0
        epoch_best_accuracy = epoch + 1
        torch.save(model.state_dict(), model_path)
        print(f"✅ Model saved with accuracy: {best_accuracy:.2%}")
    else:
        epochs_no_improve += 1

    # === Ранняя остановка ===
    if epochs_no_improve >= early_stop_epochs:
        print(f"🛑 Early stopping at epoch {epoch + 1}")
        break

# # Использование модели для предсказания
# # === 6. ПРЕДСКАЗАНИЕ С 3 КЛАССАМИ ===
# model.load_state_dict(torch.load(model_path))
# model.eval()

# last_sequence = torch.tensor(
#     df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
#     ).unsqueeze(0).to(device)

# with torch.no_grad():
#     probabilities = model(last_sequence).squeeze().tolist()
#     predicted_class = np.argmax(probabilities)

#     class_names = {0: "📉 STRONG DOWN", 1: "🔹 FLAT", 2: "📈 STRONG UP"}

#     print(f"Prediction: {class_names[predicted_class]}")
#     print(
#         f"UP Probability: {probabilities[2]:.2%}, "
#         f"FLAT Probability: {probabilities[1]:.2%}, "
#         f"DOWN Probability: {probabilities[0]:.2%}"
#         )



Epoch 1/2000, Loss: 1.0922, Test Accuracy: 38.02%, Best accuracy: 0.00%, Epoch best accuracy: 0, seed: 1
✅ Model saved with accuracy: 38.02%
Epoch 2/2000, Loss: 1.0794, Test Accuracy: 38.02%, Best accuracy: 38.02%, Epoch best accuracy: 1, seed: 1
Epoch 3/2000, Loss: 1.0784, Test Accuracy: 38.02%, Best accuracy: 38.02%, Epoch best accuracy: 1, seed: 1
Epoch 4/2000, Loss: 1.0784, Test Accuracy: 38.02%, Best accuracy: 38.02%, Epoch best accuracy: 1, seed: 1
Epoch 5/2000, Loss: 1.0783, Test Accuracy: 38.02%, Best accuracy: 38.02%, Epoch best accuracy: 1, seed: 1
Epoch 6/2000, Loss: 1.0774, Test Accuracy: 38.02%, Best accuracy: 38.02%, Epoch best accuracy: 1, seed: 1
Epoch 7/2000, Loss: 1.0764, Test Accuracy: 38.02%, Best accuracy: 38.02%, Epoch best accuracy: 1, seed: 1
Epoch 8/2000, Loss: 1.0765, Test Accuracy: 38.02%, Best accuracy: 38.02%, Epoch best accuracy: 1, seed: 1
Epoch 9/2000, Loss: 1.0756, Test Accuracy: 38.02%, Best accuracy: 38.02%, Epoch best accuracy: 1, seed: 1
Epoch 10/20

In [66]:
# === 6. ЗАГРУЗКА ЛУЧШЕЙ МОДЕЛИ И ТЕСТ ===
print("\n🔹 Loading best model for final evaluation...")
model.load_state_dict(torch.load(model_path))
model.eval()

correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch).argmax(dim=1)  # Получаем предсказанный класс

        correct += (y_pred == y_batch).sum().item()
        total += y_batch.size(0)

        all_preds.extend(y_pred.cpu().numpy())  # Сохраняем предсказания
        all_labels.extend(y_batch.cpu().numpy())  # Сохраняем реальные метки

final_accuracy = correct / total
print(f"🏆 Final Test Accuracy: {final_accuracy:.2%}")

# === 8. ДОПОЛНИТЕЛЬНАЯ СТАТИСТИКА ===
from sklearn.metrics import classification_report, confusion_matrix

print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["Падение", "Флэт", "Рост"]))

print("\n🌀 Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


🔹 Loading best model for final evaluation...
🏆 Final Test Accuracy: 39.80%

📊 Classification Report:
              precision    recall  f1-score   support

     Падение       0.37      0.24      0.29       156
        Флэт       0.40      0.79      0.53       192
        Рост       0.41      0.08      0.13       157

    accuracy                           0.40       505
   macro avg       0.40      0.37      0.32       505
weighted avg       0.40      0.40      0.33       505


🌀 Confusion Matrix:
[[ 37 110   9]
 [ 32 152   8]
 [ 30 115  12]]


🔹 Как использовать модель для предсказания?  
После загрузки модели можно предсказать направление следующей свечи:

In [67]:
# Загружаем лучшую модель
model.load_state_dict(torch.load("best_model_03.pth"))
model.eval()

# Берем последние 20 свечей из df_fut
last_sequence = torch.tensor(
    df_fut['CANDLE_INT'].iloc[-20:].values, dtype=torch.long
    ).unsqueeze(0).to(device)

# Предсказание
with torch.no_grad():
    probabilities = model(last_sequence).squeeze().tolist()  # Вероятности для 3 классов
    predicted_class = np.argmax(probabilities)  # Класс с наибольшей вероятностью

    class_names = {0: "📉 STRONG DOWN", 1: "🔹 FLAT", 2: "📈 STRONG UP"}

    print(f"Prediction: {class_names[predicted_class]}")
    print(
        f"UP Probability: {probabilities[2]:.2%}, \n"
        f"FLAT Probability: {probabilities[1]:.2%}, \n"
        f"DOWN Probability: {probabilities[0]:.2%}"
        )

Prediction: 🔹 FLAT
UP Probability: 7.16%, 
FLAT Probability: 83.30%, 
DOWN Probability: 9.54%
